# 导入必要的库

In [1]:
# 导入必要的库
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time


# 定义展示图片的函数

In [2]:
# 定义一个函数用于展示图片
def show(name,img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# 定义图像拼接函数

In [3]:
# 定义图像拼接函数
def stitch_images(image_list):
    MIN = 10
    FLANN_INDEX_KDTREE = 0
    starttime = time.time()
    # 初始化SIFT检测器
    sift = cv2.SIFT_create() 
    #创建字典
    indexParams = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    searchParams = dict(checks=50)
    flann = cv2.FlannBasedMatcher(indexParams,searchParams)

    imageA = cv2.resize(image_list[0],(0,0),fx=0.3,fy=0.3)
    for index in range(1, len(image_list)):
        imageB = cv2.resize(image_list[index],(0,0),fx=0.3,fy=0.3)
        kp1,descrip1 = sift.detectAndCompute(imageA,None)
        kp2,descrip2 = sift.detectAndCompute(imageB,None)
        match = flann.knnMatch(descrip1,descrip2,k=2)
        good = []
        #过滤特征点
        for i,(m,n) in enumerate(match):
            if(m.distance < 0.75*n.distance):
                good.append(m)
        # 当筛选后的匹配对大于10时，计算视角变换矩阵
        if len(good) > MIN:
            src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
            ano_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
            M,mask = cv2.findHomography(src_pts,ano_pts,cv2.RANSAC,5.0)
            warpImg = cv2.warpPerspective(imageB, np.linalg.inv(M), (imageA.shape[1]+imageB.shape[1], imageB.shape[0]))

            direct = warpImg.copy()
            direct[0:imageA.shape[0], 0:imageA.shape[1]] = imageA
            simple = time.time()

            rows,cols = imageA.shape[:2]
            for col in range(0,cols):
                # 开始重叠的最左端
                if imageA[:, col].any() and warpImg[:, col].any():
                    left = col
                    break
            for col in range(cols-1, 0, -1):
                #重叠的最右一列
                if imageA[:, col].any() and warpImg[:, col].any():
                    right = col
                    break
            #加权处理
            res = np.zeros([rows, cols, 3], np.uint8)
            for row in range(0, rows):
                for col in range(0, cols):
                    if not imageA[row, col].any():
                        res[row, col] = warpImg[row, col]
                    elif not warpImg[row, col].any():
                        res[row, col] = imageA[row, col]
                    else:
                        srcImgLen = float(abs(col - left))
                        testImgLen = float(abs(col - right))
                        alpha = srcImgLen / (srcImgLen + testImgLen)
                        res[row, col] = np.clip(imageA[row, col] * (1 - alpha) + warpImg[row, col] * alpha, 0, 255)

            warpImg[0:imageA.shape[0], 0:imageA.shape[1]] = res
            imageA = warpImg.copy()

    show('res',warpImg)
    final = time.time()
    print(final-starttime)


# 图像的导入和拼接

In [4]:
# 导入图片
img1 = cv2.imread('1.jpg')
img2 = cv2.imread('2.jpg')
img3 = cv2.imread('3.jpg')

# 将输入图片组成列表
image_list = [img1, img2, img3]

# 调用图像拼接函数
stitch_images(image_list)


11.51158618927002
